In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Heatmap and Layout Analysis\n",
    "\n",
    "This notebook analyzes spatial distribution of faces in classroom videos and generates heatmaps.\n",
    "\n",
    "## Features:\n",
    "- Heatmap generation (presence, attention, emotion)\n",
    "- Seat assignment analysis\n",
    "- Spatial clustering analysis\n",
    "- Classroom layout visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../src')\n",
    "\n",
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import pandas as pd\n",
    "import seaborn as sns\n",
    "from pathlib import Path\n",
    "import json\n",
    "from scipy.spatial.distance import pdist, squareform\n",
    "from sklearn.cluster import DBSCAN\n",
    "\n",
    "from config import Config\n",
    "from detection.face_tracker import FaceTracker\n",
    "from emotion.emotion_detector import EmotionDetector\n",
    "from layout_analysis.layout_mapper import LayoutMapper\n",
    "from utils.video_utils import VideoProcessor"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load configuration and initialize components\n",
    "config = Config()\n",
    "video_processor = VideoProcessor(config)\n",
    "face_tracker = FaceTracker(config)\n",
    "emotion_detector = EmotionDetector(config)\n",
    "layout_mapper = LayoutMapper(config)\n",
    "\n",
    "# Test video path\n",
    "video_path = \"../data/raw_videos/sample_classroom.mp4\"\n",
    "\n",
    "print(\"Components initialized\")\n",
    "print(f\"Video path: {video_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load or Generate Detection Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Try to load existing detection data, or generate new data\n",
    "detection_file = Path('../data/outputs/detections_opencv.json')\n",
    "\n",
    "if detection_file.exists():\n",
    "    print(\"Loading existing detection data...\")\n",
    "    with open(detection_file, 'r') as f:\n",
    "        detections = json.load(f)\n",
    "    print(f\"Loaded {len(detections)} detections\")\n",
    "else:\n",
    "    print(\"Generating new detection data...\")\n",
    "    \n",
    "    # Process video for detections\n",
    "    cap = cv2.VideoCapture(video_path)\n",
    "    if not cap.isOpened():\n",
    "        print(f\"Failed to open video: {video_path}\")\n",
    "        detections = []\n",
    "    else:\n",
    "        detections = []\n",
    "        frame_idx = 0\n",
    "        max_frames = 100\n",
    "        \n",
    "        while frame_idx < max_frames:\n",
    "            ret, frame = cap.read()\n",
    "            if not ret:\n",
    "                break\n",
    "            \n",
    "            # Detect faces\n",
    "            face_detections = face_tracker.detect_faces(frame)\n",
    "            \n",
    "            # Process each face for emotions and attention\n",
    "            for detection in face_detections:\n",
    "                bbox = detection['bbox']\n",
    "                x1, y1, x2, y2 = bbox\n",
    "                \n",
    "                # Extract face region\n",
    "                face_region = frame[y1:y2, x1:x2]\n",
    "                if face_region.size == 0:\n",
    "                    continue\n",
    "                \n",
    "                # Detect emotions and attention\n",
    "                emotions = emotion_detector.detect_emotions(face_region)\n",
    "                attention = emotion_detector.detect_attention(face_region)\n",
    "                \n",
    "                # Add to detection\n",
    "                detection.update({\n",
    "                    'frame_idx': frame_idx,\n",
    "                    'emotions': emotions,\n",
    "                    'attention': attention,\n",
    "                    'dominant_emotion': max(emotions.items(), key=lambda x: x[1])[0],\n",
    "                    'emotion_confidence': max(emotions.values()),\n",
    "                    'is_attentive': attention['attention_score'] > 0.7\n",
    "                })\n",
    "                \n",
    "                detections.append(detection)\n",
    "            \n",
    "            frame_idx += 1\n",
    "        \n",
    "        cap.release()\n",
    "        print(f\"Generated {len(detections)} detections\")\n",
    "\n",
    "if not detections:\n",
    "    print(\"No detection data available. Please ensure video file exists and contains faces.\")\n",
    "    # Create sample data for demonstration\n",
    "    detections = [\n",
    "        {\n",
    "            'bbox': [100, 100, 200, 200],\n",
    "            'frame_idx': 0,\n",
    "            'attention': {'attention_score': 0.8},\n",
    "            'emotion_confidence': 0.7,\n",
    "            'dominant_emotion': 'neutral'\n",
    "        },\n",
    "        {\n",
    "            'bbox': [300, 150, 400, 250],\n",
    "            'frame_idx': 0,\n",
    "            'attention': {'attention_score': 0.6},\n",
    "            'emotion_confidence': 0.8,\n",
    "            'dominant_emotion': 'happy'\n",
    "        }\n",
    "    ]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Heatmap Generation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Generate different types of heatmaps\n",
    "    heatmap_types = ['presence', 'attention', 'emotion']\n",
    "    heatmaps = {}\n",
    "    \n",
    "    for heatmap_type in heatmap_types:\n",
    "        heatmap = layout_mapper.generate_heatmap(detections, heatmap_type)\n",
    "        heatmaps[heatmap_type] = heatmap\n",
    "        \n",
    "        print(f\"Generated {heatmap_type} heatmap: {heatmap.shape}\")\n",
    "    \n",
    "    # Visualize heatmaps\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 6))\n",
    "    \n",
    "    for i, (heatmap_type, heatmap) in enumerate(heatmaps.items()):\n",
    "        im = axes[i].imshow(heatmap, cmap='hot', interpolation='bilinear')\n",
    "        axes[i].set_title(f'{heatmap_type.title()} Heatmap')\n",
    "        axes[i].set_xlabel('X Position')\n",
    "        axes[i].set_ylabel('Y Position')\n",
    "        plt.colorbar(im, ax=axes[i])\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Print heatmap statistics\n",
    "    print(\"\\nHeatmap Statistics:\")\n",
    "    print(\"-\" * 30)\n",
    "    for heatmap_type, heatmap in heatmaps.items():\n",
    "        print(f\"{heatmap_type.title()} Heatmap:\")\n",
    "        print(f\"  Max intensity: {heatmap.max():.3f}\")\n",
    "        print(f\"  Mean intensity: {heatmap.mean():.3f}\")\n",
    "        print(f\"  Total intensity: {heatmap.sum():.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Classroom Layout Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Visualize classroom layout with detections\n",
    "    layout_mapper.visualize_classroom_layout(detections)\n",
    "    \n",
    "    # Extract position data for analysis\n",
    "    positions = []\n",
    "    attention_scores = []\n",
    "    emotions = []\n",
    "    \n",
    "    for detection in detections:\n",
    "        if 'bbox' in detection:\n",
    "            bbox = detection['bbox']\n",
    "            x1, y1, x2, y2 = bbox\n",
    "            center_x = (x1 + x2) / 2\n",
    "            center_y = (y1 + y2) / 2\n",
    "            \n",
    "            positions.append([center_x, center_y])\n",
    "            attention_scores.append(detection.get('attention', {}).get('attention_score', 0.5))\n",
    "            emotions.append(detection.get('dominant_emotion', 'neutral'))\n",
    "    \n",
    "    if positions:\n",
    "        positions = np.array(positions)\n",
    "        \n",
    "        # Create detailed layout analysis\n",
    "        fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n",
    "        \n",
    "        # Position scatter plot with attention coloring\n",
    "        scatter = axes[0, 0].scatter(positions[:, 0], positions[:, 1], \n",
    "                                    c=attention_scores, cmap='RdYlGn', s=100, alpha=0.7)\n",
    "        axes[0, 0].set_title('Face Positions (colored by attention)')\n",
    "        axes[0, 0].set_xlabel('X Position (pixels)')\n",
    "        axes[0, 0].set_ylabel('Y Position (pixels)')\n",
    "        axes[0, 0].grid(True, alpha=0.3)\n",
    "        plt.colorbar(scatter, ax=axes[0, 0])\n",
    "        \n",
    "        # Position distribution\n",
    "        axes[0, 1].hist2d(positions[:, 0], positions[:, 1], bins=20, cmap='Blues')\n",
    "        axes[0, 1].set_title('Position Density')\n",
    "        axes[0, 1].set_xlabel('X Position (pixels)')\n",
    "        axes[0, 1].set_ylabel('Y Position (pixels)')\n",
    "        \n",
    "        # X position distribution\n",
    "        axes[1, 0].hist(positions[:, 0], bins=20, alpha=0.7, color='skyblue')\n",
    "        axes[1, 0].set_title('X Position Distribution')\n",
    "        axes[1, 0].set_xlabel('X Position (pixels)')\n",
    "        axes[1, 0].set_ylabel('Frequency')\n",
    "        \n",
    "        # Y position distribution\n",
    "        axes[1, 1].hist(positions[:, 1], bins=20, alpha=0.7, color='lightcoral')\n",
    "        axes[1, 1].set_title('Y Position Distribution')\n",
    "        axes[1, 1].set_xlabel('Y Position (pixels)')\n",
    "        axes[1, 1].set_ylabel('Frequency')\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Spatial Clustering Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections and len(detections) > 1:\n",
    "    # Extract positions for clustering\n",
    "    positions = []\n",
    "    for detection in detections:\n",
    "        if 'bbox' in detection:\n",
    "            bbox = detection['bbox']\n",
    "            x1, y1, x2, y2 = bbox\n",
    "            center_x = (x1 + x2) / 2\n",
    "            center_y = (y1 + y2) / 2\n",
    "            positions.append([center_x, center_y])\n",
    "    \n",
    "    if len(positions) > 1:\n",
    "        positions = np.array(positions)\n",
    "        \n",
    "        # Perform clustering\n",
    "        clustering = DBSCAN(eps=100, min_samples=2).fit(positions)\n",
    "        labels = clustering.labels_\n",
    "        \n",
    "        # Calculate spatial statistics\n",
    "        distances = pdist(positions)\n",
    "        avg_distance = np.mean(distances)\n",
    "        min_distance = np.min(distances)\n",
    "        max_distance = np.max(distances)\n",
    "        \n",
    "        # Visualize clustering results\n",
    "        fig, axes = plt.subplots(1, 2, figsize=(15, 6))\n",
    "        \n",
    "        # Clustering visualization\n",
    "        unique_labels = set(labels)\n",
    "        colors = plt.cm.Set3(np.linspace(0, 1, len(unique_labels)))\n",
    "        \n",
    "        for k, col in zip(unique_labels, colors):\n",
    "            if k == -1:\n",
    "                # Black used for noise\n",
    "                col = 'black'\n",
    "            \n",
    "            class_member_mask = (labels == k)\n",
    "            xy = positions[class_member_mask]\n",
    "            axes[0].plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, \n",
    "                        markeredgecolor='k', markersize=8, label=f'Cluster {k}')\n",
    "        \n",
    "        axes[0].set_title('Spatial Clustering of Face Positions')\n",
    "        axes[0].set_xlabel('X Position (pixels)')\n",
    "        axes[0].set_ylabel('Y Position (pixels)')\n",
    "        axes[0].legend()\n",
    "        axes[0].grid(True, alpha=0.3)\n",
    "        \n",
    "        # Distance distribution\n",
    "        axes[1].hist(distances, bins=20, alpha=0.7, color='green')\n",
    "        axes[1].axvline(avg_distance, color='red', linestyle='--', \n",
    "                        label=f'Mean: {avg_distance:.1f}')\n",
    "        axes[1].set_title('Distance Distribution Between Faces')\n",
    "        axes[1].set_xlabel('Distance (pixels)')\n",
    "        axes[1].set_ylabel('Frequency')\n",
    "        axes[1].legend()\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "        \n",
    "        # Print clustering statistics\n",
    "        print(\"\\nSpatial Clustering Analysis:\")\n",
    "        print(\"-\" * 40)\n",
    "        print(f\"Number of clusters: {len(unique_labels) - (1 if -1 in labels else 0)}\")\n",
    "        print(f\"Noise points: {list(labels).count(-1)}\")\n",
    "        print(f\"Average distance between faces: {avg_distance:.1f} pixels\")\n",
    "        print(f\"Minimum distance: {min_distance:.1f} pixels\")\n",
    "        print(f\"Maximum distance: {max_distance:.1f} pixels\")\n",
    "        \n",
    "        # Cluster details\n",
    "        for label in sorted(unique_labels):\n",
    "            if label != -1:\n",
    "                cluster_size = list(labels).count(label)\n",
    "                cluster_positions = positions[labels == label]\n",
    "                cluster_center = np.mean(cluster_positions, axis=0)\n",
    "                print(f\"\\nCluster {label}:\")\n",
    "                print(f\"  Size: {cluster_size} faces\")\n",
    "                print(f\"  Center: ({cluster_center[0]:.1f}, {cluster_center[1]:.1f})\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Seat Assignment Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Assign seats to detected faces\n",
    "    seat_assignments = layout_mapper.assign_seats(detections)\n",
    "    \n",
    "    if seat_assignments:\n",
    "        print(f\"\\nSeat Assignments ({len(seat_assignments)} assigned):\")\n",
    "        print(\"-\" * 50)\n",
    "        \n",
    "        for seat_id, assignment in seat_assignments.items():\n",
    "            print(f\"{seat_id}: {assignment['identity']} \")\n",
    "            print(f\"  Attention: {assignment['attention_score']:.3f}\")\n",
    "            print(f\"  Emotion: {assignment['dominant_emotion']}\")\n",
    "            print(f\"  Position: ({assignment['position'][0]:.1f}, {assignment['position'][1]:.1f})\")\n",
    "        \n",
    "        # Analyze seat assignments\n",
    "        attention_by_seat = {seat: data['attention_score'] for seat, data in seat_assignments.items()}\n",
    "        emotions_by_seat = {seat: data['dominant_emotion'] for seat, data in seat_assignments.items()}\n",
    "        \n",
    "        # Create seat analysis visualization\n",
    "        fig, axes = plt.subplots(1, 2, figsize=(15, 6))\n",
    "        \n",
    "        # Attention by seat\n",
    "        seats = list(attention_by_seat.keys())\n",
    "        attention_values = list(attention_by_seat.values())\n",
    "        \n",
    "        bars1 = axes[0].bar(seats, attention_values, color='lightblue')\n",
    "        axes[0].set_title('Attention Scores by Seat')\n",
    "        axes[0].set_xlabel('Seat')\n",
    "        axes[0].set_ylabel('Attention Score')\n",
    "        axes[0].tick_params(axis='x', rotation=45)\n",
    "        \n",
    "        # Add value labels on bars\n",
    "        for bar, value in zip(bars1, attention_values):\n",
    "            axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n",
    "                        f'{value:.2f}', ha='center', va='bottom')\n",
    "        \n",
    "        # Emotion by seat\n",
    "        emotion_counts = Counter(emotions_by_seat.values())\n",
    "        axes[1].pie(emotion_counts.values(), labels=emotion_counts.keys(), autopct='%1.1f%%')\n",
    "        axes[1].set_title('Emotion Distribution by Seat')\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "        \n",
    "        # Print seat statistics\n",
    "        print(\"\\nSeat Assignment Statistics:\")\n",
    "        print(\"-\" * 40)\n",
    "        print(f\"Average attention score: {np.mean(attention_values):.3f}\")\n",
    "        print(f\"Highest attention: {max(attention_values):.3f} ({seats[attention_values.index(max(attention_values))]})\")\n",
    "        print(f\"Lowest attention: {min(attention_values):.3f} ({seats[attention_values.index(min(attention_values))]})\")\n",
    "        \n",
    "        print(\"\\nEmotion distribution:\")\n",
    "        for emotion, count in emotion_counts.most_common():\n",
    "            print(f\"  {emotion.title()}: {count}\")\n",
    "    else:\n",
    "        print(\"No seat assignments made\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Spatial Distribution Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Perform comprehensive spatial analysis\n",
    "    spatial_analysis = layout_mapper.analyze_spatial_distribution(detections)\n",
    "    \n",
    "    if spatial_analysis:\n",
    "        print(\"\\nSpatial Distribution Analysis:\")\n",
    "        print(\"=\" * 50)\n",
    "        \n",
    "        print(f\"Total faces detected: {spatial_analysis['total_faces']}\")\n",
    "        print(f\"Overall attention score: {spatial_analysis['overall_attention']:.3f}\")\n",
    "        \n",
    "        print(\"\\nAttention by Region:\")\n",
    "        for region, score in spatial_analysis['attention_by_region'].items():\n",
    "            print(f\"  {region.capitalize()}: {score:.3f}\")\n",
    "        \n",
    "        print(\"\\nEmotion Distribution:\")\n",
    "        for emotion, count in spatial_analysis['emotion_distribution'].items():\n",
    "            print(f\"  {emotion.title()}: {count}\")\n",
    "        \n",
    "        print(f\"\\nSpatial Statistics:\")\n",
    "        print(f\"  Average distance between faces: {spatial_analysis['average_distance']:.1f} pixels\")\n",
    "        print(f\"  Minimum distance between faces: {spatial_analysis['minimum_distance']:.1f} pixels\")\n",
    "        \n",
    "        # Visualize regional analysis\n",
    "        regions = list(spatial_analysis['attention_by_region'].keys())\n",
    "        attention_scores = list(spatial_analysis['attention_by_region'].values())\n",
    "        \n",
    "        plt.figure(figsize=(10, 6))\n",
    "        bars = plt.bar(regions, attention_scores, color=['red', 'blue', 'green', 'orange'])\n",
    "        plt.title('Attention Scores by Classroom Region')\n",
    "        plt.xlabel('Region')\n",
    "        plt.ylabel('Average Attention Score')\n",
    "        \n",
    "        # Add value labels on bars\n",
    "        for bar, score in zip(bars, attention_scores):\n",
    "            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n",
    "                    f'{score:.3f}', ha='center', va='bottom')\n",
    "        \n",
    "        plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Save Analysis Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save all analysis results\n",
    "output_path = Path('../data/outputs')\n",
    "output_path.mkdir(parents=True, exist_ok=True)\n",
    "\n",
    "# Save heatmaps\n",
    "if 'heatmaps' in locals() and heatmaps:\n",
    "    for heatmap_type, heatmap in heatmaps.items():\n",
    "        np.save(output_path / f'heatmap_{heatmap_type}.npy', heatmap)\n",
    "    print(f\"Heatmaps saved to {output_path}\")\n",
    "\n",
    "# Save seat assignments\n",
    "if 'seat_assignments' in locals() and seat_assignments:\n",
    "    with open(output_path / 'seat_assignments.json', 'w') as f:\n",
    "        json.dump(seat_assignments, f, indent=2, default=str)\n",
    "    print(f\"Seat assignments saved to {output_path / 'seat_assignments.json'}\")\n",
    "\n",
    "# Save spatial analysis\n",
    "if 'spatial_analysis' in locals() and spatial_analysis:\n",
    "    with open(output_path / 'spatial_analysis.json', 'w') as f:\n",
    "        json.dump(spatial_analysis, f, indent=2)\n",
    "    print(f\"Spatial analysis saved to {output_path / 'spatial_analysis.json'}\")\n",
    "\n",
    "# Generate comprehensive report\n",
    "if detections:\n",
    "    layout_mapper.generate_report(detections, str(output_path))\n",
    "    print(f\"Comprehensive report generated in {output_path}\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}